In [ ]:
#@title נא לבחור פרמטרים מתאימים  { run: "auto" }

threshold_type = "Ambient Residential" # @param ["Ambient Residential","Ambient Industrial"]

file_name = 'ניסיון תוך מבני' # @param {type:"string"}

Font_size_hebrew = 9 # @param {type:"slider", min:6, max:24, step:1}

Font_name_hebrew = 'David' # @param {type:"string"}

Font_size_english = 8 # @param {type:"slider", min:6, max:24, step:1}

Font_name_english = 'Times New Roman' # @param {type:"string"}

!pip install openpyxl
from openpyxl.styles import Font
import pandas as pd
import re

# Define the full dataset based on the content of your CSV file
data = {
    'חומר נבדק': [
        '1,1 DiChloroEthane', '1,1 DichloroEthene', '1,1,1-trichloroEthane',
        '1,1,2-trichloro-1,2,2-trifluoro-Ethane', '1,2,4-trimethylBenzene',
        '1,3,5-TriMethylBenzene', 'Benzene', 'ChloroBenzene',
        'Ethylbenzene', 'Isopropanol', 'MTBE', 'Naphthalene',
        'Xylene', 'Tetrachloroethene', 'Toluene'
    ],
    'CAS': [
        '75-34-3', '75-35-4', '71-55-6', '76-13-1', '95-63-6',
        '108-67-8', '71-43-2', '108-90-7', '100-41-4', '67-63-0',
        '1634-04-4', '91-20-3', '95-47-6', '127-18-4', '108-88-3'
    ],
    'LOD [µg/m^3]': [
        0.81, 0.79, 1.09, 1.09, 1.2,
        0.44, 0.64, 0.92, 0.87, 0.49,
        0.82, 1.05, 1.40, 1.36, 0.75
    ],
    'LOQ [µg/m^3]': [
        4.05, 3.96, 5.46, 5.46, 6.01,
        2.21, 3.19, 4.6, 4.34, 2.46,
        4.1, 5.24, 4.20, 6.78, 3.77
    ],
    'ערך סף': [None] * 15
}

# Create the DataFrame
df_template = pd.DataFrame(data)



In [ ]:
#@title להעלות קובץ ערכי סף של גז קרקע בלחיצה על הכפתור למטה
from google.colab import files
uploaded = files.upload()

filenames = list(uploaded.keys())

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through each file and read it based on its extension
for filename in filenames:
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
    elif filename.endswith('.xlsx'):
        df = pd.read_excel(filename)
    else:
        print(f"Unsupported file format: {filename}")
        continue
        # Round all numeric cells to 2 decimal points
    df = df.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)

    dataframes.append(df)

# Concatenate all dataframes into one
if dataframes:
    threshold_data = pd.concat(dataframes, ignore_index=True)
else:
    print("No valid files uploaded.")


Saving גרסה 7 דצמבר 2024 - AvirTochMivni_threshold.csv to גרסה 7 דצמבר 2024 - AvirTochMivni_threshold.csv


<ipython-input-3-1142315985>:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)


In [ ]:
#@title להעלות קבצי תוצאות של אלכם בלחיצה על הכפתור למטה
from google.colab import files
uploaded = files.upload()

filenames = list(uploaded.keys())

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through each file and read it based on its extension
for filename in filenames:
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
    elif filename.endswith('.xlsx'):
        df = pd.read_excel(filename)
    else:
        print(f"Unsupported file format: {filename}")
        continue

            # Round all numeric cells to 2 decimal points
    df = df.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)
    dataframes.append(df)

# Concatenate all dataframes into one
if dataframes:
    results_data_raw = pd.concat(dataframes, ignore_index=True)
else:
    print("No valid files uploaded.")


# Convert the data to string to avoid errors
results_data = results_data_raw.astype(str)

# Find the rows that contain "Analysis Location:"
location_row_index = None
location_headers = []

for i, row in results_data.iterrows():
    if row.str.contains('Analysis Location:', na=False).any():
        location_row_index = i
        break

# If the row with "Analysis Location:" is found, extract location names
if location_row_index is not None and location_row_index + 1 < len(results_data):
    location_row = results_data.iloc[location_row_index]
    next_row = results_data.iloc[location_row_index + 1]

    for j, cell in enumerate(location_row):
        if pd.notna(cell) and next_row.iloc[j] == "Final Conc. [ug/m^3]":
            location_headers.append(cell)
            # Assign a single string value instead of a list
            results_data.iloc[location_row_index + 1, j] = ', '.join(location_headers[-1:])

# Adding the columns for each location to df_template
for location in location_headers:
    df_template[location] = None


# Set the headers to be the values from row number 2 (index 2 in zero-based indexing)
results_data.columns = results_data.iloc[2]

# Drop the first three rows as they are now part of the headers
results_data = results_data.drop([0, 1, 2]).reset_index(drop=True)



# Ensure that both CAS columns are treated as strings
df_template['CAS'] = df_template['CAS'].astype(str)
results_data['CAS'] = results_data['CAS'].astype(str)
threshold_data['CAS No.'] = threshold_data['CAS No.'].astype(str)

# Add 'ערך סף' column to results_data
results_data['ערך סף'] = None

# Iterate through each row in df_template
for i, row in df_template.iterrows():
    # Find the matching row in the threshold_data DataFrame
    matching_row = threshold_data[threshold_data['CAS No.'] == row['CAS']]

    # If a matching row is found, update the df_template with the value from the `type` column
    if not matching_row.empty:
        value = matching_row.iloc[0][threshold_type]
        if pd.notna(value):  # Check if the value is not NaN
            df_template.at[i, 'ערך סף'] = round(value, 2)  # Round to 2 decimal places

# Now, iterate through each row in results_data
for i, row in results_data.iterrows():
    # Find the matching row in the threshold_data DataFrame
    matching_row = threshold_data[threshold_data['CAS No.'] == row['CAS']]

    # If a matching row is found, update the results_data with the value from the `type` column
    if not matching_row.empty:
        value = matching_row.iloc[0][threshold_type]
        if pd.notna(value):  # Check if the value is not NaN
            results_data.at[i, 'ערך סף'] = round(value, 3)  # Round to 2 decimal places

# Iterate through each row in df_template
for i, template_row in df_template.iterrows():
    # Get the CAS number from the current row of df_template
    template_cas = template_row['CAS'].strip()

    # Find the corresponding row in results_data based on the CAS number
    matching_row = results_data[results_data['CAS'].str.strip() == template_cas]

    # If a matching row is found, fill the values for each location header
    if not matching_row.empty:
        matching_row = matching_row.iloc[0]  # Assuming CAS numbers are unique, so we take the first match
        for location in location_headers:
            if location in df_template.columns:
                df_template.at[i, location] = matching_row[location]

# Create a set to track CAS numbers that are already in df_template
existing_cas_numbers = set(df_template['CAS'])

# Create a list to track new CAS numbers being added
newly_added_cas_numbers = set()

# Create a list to store the new rows
new_rows = []

# Iterate through each row in results_data
for i, row in results_data.iterrows():
    cas_number = row['CAS']
    sff_value = pd.to_numeric(row['ערך סף'], errors='coerce')
    lod_value = pd.to_numeric(row['LOD [ug/m^3]'], errors='coerce')
    loq_value = pd.to_numeric(row['LOQ. [ug/m^3]'], errors='coerce')

    # Skip if ערך סף is NaN or if the CAS number already exists in df_template
    if pd.isna(sff_value) or cas_number in existing_cas_numbers or cas_number in newly_added_cas_numbers:
        continue

    exceeded = False  # Flag to check if we need to add a new row
    new_row = {
        'חומר נבדק': row['Name'],
        'CAS': cas_number,
        'LOD [µg/m^3]': row['LOD [ug/m^3]'],
        'LOQ [µg/m^3]': row['LOQ. [ug/m^3]'],
        'ערך סף': sff_value
    }

    # Iterate over location headers to check conditions
    for location in location_headers:
        cell_value = row[location]
        numeric_value = pd.to_numeric(cell_value, errors='coerce')

        # Case 1: Location value exceeds ערך סף
        if numeric_value > sff_value:
            new_row[location] = numeric_value
            exceeded = True

        # Case 2: Cell contains "N.D." and LOD [µg/m^3] > ערך סף
        elif isinstance(cell_value, str) and cell_value.strip() == "N.D.":
            if lod_value > sff_value:
                new_row[location] = f"<{lod_value}"  # Keep format as "<LOD value>"
                exceeded = True

        # Case 3: Cell contains "<LOQ" and LOQ [µg/m^3] > ערך סף
        elif isinstance(cell_value, str) and cell_value.strip() == "<LOQ":
            if loq_value > sff_value:
                new_row[location] = f"<{loq_value}"  # Keep format as "<LOQ value>"
                exceeded = True

    # Only add the row if at least one condition was met
    if exceeded:
        new_rows.append(new_row)
        newly_added_cas_numbers.add(cas_number)  # Track CAS number to avoid duplicates

# Append the new rows to df_template if any exist
if new_rows:
    df_template = pd.concat([df_template, pd.DataFrame(new_rows)], ignore_index=True)


for i, row in df_template.iterrows():
    for col in df_template.columns:
        if row[col] == "N.D.":
            # Replace "N.D." with "<LOD [µg/m^3]" value
            df_template.at[i, col] = f"<{row['LOD [µg/m^3]']}"
        elif row[col] == "<LOQ":
            # Replace "<LOQ" with "<LOQ [µg/m^3]" value
            df_template.at[i, col] = f"<{row['LOQ [µg/m^3]']}"

from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, PatternFill, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from google.colab import files

# Create an Excel workbook and add a worksheet
wb = Workbook()
ws = wb.active
ws.sheet_view.rightToLeft = True  # Set direction

# Define styles
thin_border = Border(
    left=Side(style='thin'), right=Side(style='thin'),
    top=Side(style='thin'), bottom=Side(style='thin')
)
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
gray_fill = PatternFill(start_color="D3D3D3", end_color="D3D3D3", fill_type="solid")
bold_font = Font(bold=True)  # Bold font for relevant cells
alignment_center_wrap = Alignment(horizontal='center', vertical='center', wrap_text=True)  # Enable text wrapping

# **Step 1: Add Table Headers and Data**
header_row = list(df_template.columns)

for col_idx, col_name in enumerate(header_row[5:], start=6):  # Start at column 6
    cell = ws.cell(row=1, column=col_idx, value=col_name)
    cell.alignment = alignment_center_wrap
    cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)

# **Extra rows for merged headers in the first 5 columns**
ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=5)
ws.cell(row=1, column=1, value="שם קידוח").alignment = alignment_center_wrap
ws.cell(row=1, column=1).font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)

ws.merge_cells(start_row=2, start_column=1, end_row=2, end_column=5)
ws.cell(row=2, column=1, value="תאריך ביצוע הדיגום").alignment = alignment_center_wrap
ws.cell(row=2, column=1).font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)

ws.merge_cells(start_row=3, start_column=1, end_row=3, end_column=5)
ws.cell(row=3, column=1, value='קריאות PID משפת הבאר [חל"מ]').alignment = alignment_center_wrap
ws.cell(row=3, column=1).font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)

# **Merge Row 4 from Column 6+ and Write "Final Conc. [µg/m^3]"**
ws.merge_cells(start_row=4, start_column=6, end_row=4, end_column=len(header_row))
ws.cell(row=4, column=6, value="Final Conc. [µg/m^3]").alignment = alignment_center_wrap
ws.cell(row=4, column=6).font = Font(name=Font_name_english, size=Font_size_english, bold=True)

# **Write Table Headers in Row 4 (Columns 1-5)**
for col_idx, col_name in enumerate(header_row[:5], start=1):  # Only first 5 columns
    ws.cell(row=4, column=col_idx, value=col_name).alignment = alignment_center_wrap
    ws.cell(row=4, column=col_idx).font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)

# **Step 2: Write Data Starting from Row 5**
# **Excel Formatting - Apply Gray Fill for LOQ Condition**
for r_idx, row in enumerate(dataframe_to_rows(df_template, index=False, header=False), 5):  # Start from row 5
    ws.append(row)

    for loc in location_headers:
        col_idx = df_template.columns.get_loc(loc) + 1  # Convert to openpyxl 1-based index
        cell = ws.cell(row=r_idx, column=col_idx)
        cell_value = row[col_idx - 1]  # Adjust for 0-indexing

        # Get the ערך סף, LOD, and LOQ values
        sff_value = pd.to_numeric(row[df_template.columns.get_loc('ערך סף')], errors='coerce')
        lod_value = pd.to_numeric(row[df_template.columns.get_loc('LOD [µg/m^3]')], errors='coerce')
        loq_value = pd.to_numeric(row[df_template.columns.get_loc('LOQ [µg/m^3]')], errors='coerce')
        numeric_value = pd.to_numeric(cell_value, errors='coerce')

        # **Yellow Fill for Numeric Values Exceeding ערך סף**
        if numeric_value > sff_value:
            cell.fill = yellow_fill
            cell.font = bold_font  # Bold text

        # **Gray Fill for "N.D." cases where LOD [µg/m^3] > ערך סף**
        elif isinstance(cell_value, str) and cell_value.startswith("<"):
            extracted_value = pd.to_numeric(cell_value.replace("<", ""), errors='coerce')
            if extracted_value == lod_value and lod_value > sff_value:
                cell.fill = gray_fill
                cell.font = bold_font
            elif extracted_value == loq_value and loq_value > sff_value:
                cell.fill = gray_fill
                cell.font = bold_font

# **Step 3: Apply Borders to the Table**
for row in ws.iter_rows(min_row=1, max_row=len(df_template) + 4, min_col=1, max_col=len(df_template.columns)):
    for cell in row:
        cell.border = thin_border  # Ensure table keeps its borders


# **Step 4: Add Legend Below the Table (Borders Only Around Legend Cells)**
start_col = 1  # Legend starts from the first column
legend_start_row = len(df_template) + 6  # Determine the row number after the table

# **Yellow Legend: "חורג מערך סף"**
legend_vsl_cell = ws.cell(row=legend_start_row, column=start_col)
legend_vsl_cell.value = "חורג מערך סף"
legend_vsl_cell.font = bold_font
legend_vsl_cell.alignment = alignment_center_wrap
legend_vsl_cell.fill = yellow_fill
legend_vsl_cell.border = thin_border

# **Gray Legend: "ערך סף גילוי מעבדה גדול מערך סף"**
legend_gray_cell = ws.cell(row=legend_start_row + 1, column=start_col)
legend_gray_cell.value = "ערך סף גילוי מעבדה גדול מערך סף"
legend_gray_cell.font = bold_font
legend_gray_cell.alignment = alignment_center_wrap
legend_gray_cell.fill = gray_fill
legend_gray_cell.border = thin_border

# **Apply font and alignment to all cells**
for row in ws.iter_rows():
    for cell in row:
        if any(char.isalpha() and char.isascii() for char in str(cell.value)):
            cell.font = Font(name=Font_name_english, size=Font_size_english)
        else:
            cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew)

        cell.alignment = Alignment(horizontal='center', vertical='center')

# **Ensure Only Legend Cells Have Borders & Wrap Text**
for cell in [legend_vsl_cell, legend_gray_cell]:
    cell.border = Border(
        left=Side(style="thin"), right=Side(style="thin"),
        top=Side(style="thin"), bottom=Side(style="thin")
    )
    cell.alignment = alignment_center_wrap  # Ensure wrap text for legend

# Save the final Excel file
file_path = f'{file_name}.xlsx'
wb.save(file_path)
files.download(file_path)


Saving דוגמת תוצאות מעבדה.xlsx to דוגמת תוצאות מעבדה.xlsx


<ipython-input-4-999535435>:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>